In [35]:
import os
adir = '../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/'
project_name = 'immune.combined'
gcs = False
results_dir = '../../../results'
abase = adir

In [36]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
os.path.dirname(sys.executable)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/usr/bin'

In [37]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import re
import matplotlib.gridspec as gridspec

## Functions for reading files straight from google cloud storage
import sys
sys.path.append('.')

from ut import read_adata, save_adata

from cnmf import get_highvar_genes_sparse, save_df_to_npz

import palettable
from IPython.display import display, Image

from joblib import parallel_backend

In [38]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(scanpy=True, dpi=80, vector_friendly=False, ipython_format='png')

In [41]:
processed_file = os.path.join(adir, "03_immune.combined.ready.h5ad")
adata_annot = read_adata(processed_file, gcs=gcs)

Only considering the two last: ['.ready', '.h5ad'].
Only considering the two last: ['.ready', '.h5ad'].


AnnData object with n_obs × n_vars = 48225 × 15358
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'ident', 'total_features_by_counts', 'log10_total_features_by_counts', 'total_counts', 'log10_total_counts', 'pct_counts_in_top_50_features', 'pct_counts_in_top_100_features', 'pct_counts_in_top_200_features', 'pct_counts_in_top_500_features', 'percent.mt', 'individual', 'cond', 'dpi', 'batch', 'analysis', 'sample', 'hpi', 'RNA_snn_res.0.5', 'seurat_clusters'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_pca', 'X_umap'

In [52]:
adata = sc.AnnData(adata_annot.X, obs=adata_annot.obs, var=adata_annot.var, uns=adata_annot.uns)
sc.pp.filter_genes(adata, min_cells=10)
adata = adata.copy()

filtered out 12 genes that are detected in less than 10 cells


In [63]:
cnmf_dir = adir

if not os.path.exists(cnmf_dir):
    os.mkdir(cnmf_dir)
    
name = 'cNMF.{project}'.format(project="03_immune.combined")
count_fn = os.path.join(cnmf_dir, name+ '.h5ad')
count_fn_npz = os.path.join(cnmf_dir, name+ '.npz')

hvg_fn = os.path.join(cnmf_dir, name+ '.hvgs.txt')
print(count_fn)
print(count_fn_npz)
print(hvg_fn)

../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/cNMF.03_immune.combined.h5ad
../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/cNMF.03_immune.combined.npz
../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/cNMF.03_immune.combined.hvgs.txt


In [64]:
sc.write(count_fn, adata)

Only considering the two last: ['.combined', '.h5ad'].
Only considering the two last: ['.combined', '.h5ad'].


In [65]:
X = pd.DataFrame(adata.X.todense(), index=adata.obs.index, columns=adata.var.index)
save_df_to_npz(X, count_fn_npz)

# Picking high-variance genes excluding the cell cycle and freeze-thaw genes associated genes so they won't drive the usage inference but keeping them in the full data matrix so that their contribution will be estimated in the final set of GEPs¶
This is running the cNMF hvg selection pipeline

In [66]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=(10**6))

normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [67]:
gene_stats, _ = get_highvar_genes_sparse(adata.X, expected_fano_threshold=None,
                       minimal_mean=0.01, numgenes=2000)

In [68]:
hvgs = list(adata.var.index[gene_stats['high_var'].values])


In [69]:
open(hvg_fn, 'w').write('\n'.join(hvgs))


25512

In [70]:
numiter=20
numworkers=50
K = ' '.join([str(i) for i in range(10,25)])
seed = 14

In [71]:
prepare_cmd = '/opt/miniconda3/bin/activate cnmf_env && python ../../Code/cNMF/cnmf.py prepare --output-dir %s --name %s -c %s -k %s --n-iter %d --total-workers %d --seed %d --beta-loss frobenius --genes-file %s' % (cnmf_dir, name, count_fn_npz, K, numiter, numworkers, seed,  hvg_fn)
print(prepare_cmd)
#! {prepare_cmd}

/opt/miniconda3/bin/activate cnmf_env && python ../../Code/cNMF/cnmf.py prepare --output-dir ../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/ --name cNMF.03_immune.combined -c ../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/cNMF.03_immune.combined.npz -k 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 --n-iter 20 --total-workers 50 --seed 14 --beta-loss frobenius --genes-file ../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/cNMF.03_immune.combined.hvgs.txt


In [72]:
kselect_plot_cmd = 'python ../../Code/cNMF/cnmf.py k_selection_plot --output-dir %s --name %s' % (cnmf_dir, name)
print('K selection plot command: %s' % kselect_plot_cmd)
#!{kselect_plot_cmd}

K selection plot command: python ../../Code/cNMF/cnmf.py k_selection_plot --output-dir ../../../../../../data/02_scRNA-Seq_PBMCs/00_scRNA-Seq_exVivo_rhemac10/05_RObjectsOLD/03_prep/ --name cNMF.03_immune.combined
python: can't open file '../../Code/cNMF/cnmf.py': [Errno 2] No such file or directory
